In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from vimco import *
from plots import *
from torch.distributions.dirichlet import Dirichlet
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [14]:
T = 30
K = 3
D = 2

## Model Parameters
num_samples = 20
num_particles_smc = 100
NUM_HIDDEN = 64
NUM_LATENTS = K*K
NUM_OBS =  2 * D
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-4
CUDA = False
PATH_ENC = "baseline-samples=%d, smc_samples=%d, T=%d, BATCH=%d" % (num_samples, num_particles_smc, T, BATCH_SIZE)

In [9]:
Ys = torch.from_numpy(np.load('dataset/sequences.npy')).float()
As_true = torch.from_numpy(np.load('dataset/transitions.npy')).float()
Zs_true = torch.from_numpy(np.load('dataset/states.npy')).float()
mus_true = torch.from_numpy(np.load('dataset/means.npy')).float()
covs_true = torch.from_numpy(np.load('dataset/covariances.npy')).float()
Pi = torch.from_numpy(np.load('dataset/init.npy')).float()

prior = initial_trans_prior(K)

num_seqs = Ys.shape[0]

Ys_Zs = np.concatenate((Ys, Zs_true), axis=-1)

In [10]:
class Encoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.latent_dir = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, prior, batch_size):
        As = torch.zeros((batch_size, K, K))
        hidden = self.enc_hidden(obs)
        alphas = F.softmax(self.latent_dir(hidden), -1).view(batch_size, T-1, K*K).sum(1).view(batch_size, K, K) + prior
        for k in range(K):
            As[:, k, :] = Dirichlet(alphas[:, k, :]).sample()
        return alphas, As

In [11]:
def initialize():
    enc = Encoder()
    if CUDA:
        enc.cuda()
    optimizer =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE)    
    return enc, optimizer
enc, optimizer = initialize()

In [12]:
ELBOs = []
KLs = []
Grads = []
num_seqs = Ys.shape[0]
prior_mcmc = torch.ones((K, K)).float()
for k in range(K):
    prior_mcmc[k, k] = 2.0
    
Grad_Steps = int((Ys.shape[0] / BATCH_SIZE))
for epoch in range(NUM_EPOCHS):
    
    indices = torch.randperm(num_seqs)
    for step in range(Grad_Steps):
        time_start = time.time()
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        batch_data = Ys[batch_indices]
        batch_zs = Zs_true[batch_indices]
        
        gradient, elbo,kl = baseline_vimco(enc, prior_mcmc, batch_zs, Pi, mus_true, covs_true, batch_data, T, D, K, num_samples, num_particles_smc, BATCH_SIZE)
     
        loss = - gradient
        loss.backward()
        optimizer.step()
        ELBOs.append(elbo.item())
        KLs.append(kl.item())
        Grads.append(loss.item())
        time_end = time.time()
        print('iteration : %d, ELBO : %f, KL : %f (%ds)' % (epoch*BATCH_SIZE+step, elbo, kl, time_end - time_start))

iteration : 0, ELBO : -133.074265, KL : 12.476903 (22s)
iteration : 1, ELBO : -129.892441, KL : 10.956992 (21s)
iteration : 2, ELBO : -132.596985, KL : 12.079511 (20s)
iteration : 3, ELBO : -130.985687, KL : 12.249022 (21s)
iteration : 4, ELBO : -132.491241, KL : 16.016489 (21s)
iteration : 5, ELBO : -130.868805, KL : 11.282068 (21s)
iteration : 6, ELBO : -127.115982, KL : 15.126233 (21s)
iteration : 7, ELBO : -128.502716, KL : 12.962898 (22s)
iteration : 8, ELBO : -133.383453, KL : 13.971252 (22s)
iteration : 9, ELBO : -131.296967, KL : 12.917890 (21s)
iteration : 10, ELBO : -131.528595, KL : 13.396196 (21s)
iteration : 11, ELBO : -132.073944, KL : 13.519419 (21s)
iteration : 12, ELBO : -131.281647, KL : 13.035153 (21s)
iteration : 13, ELBO : -128.370148, KL : 10.611334 (21s)
iteration : 14, ELBO : -131.422211, KL : 12.818980 (21s)
iteration : 15, ELBO : -133.209717, KL : 12.795574 (21s)
iteration : 16, ELBO : -130.649994, KL : 12.531443 (19s)
iteration : 17, ELBO : -130.571518, KL : 

KeyboardInterrupt: 

In [7]:
save_params(ELBOs, KLs, PATH_ENC)

torch.Size([500, 30, 3])